In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row



In [ ]:
import os

In [ ]:
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("myApp") \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.sql.catalogImplementation", "hive")\
    .getOrCreate()

spark.conf.set("spark.sql.ansi.enabled", "false")

spark dashboard is available at: [http://localhost:4040/](http://localhost:4040/)

In [ ]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

In [ ]:
df.show()

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

## Create a Dataframe from pandas

In [ ]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.describe()

## Selecting and Accessing Data

In [ ]:
df.a

In [ ]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

In [ ]:
type(df.c) == type(upper(df.c)) == type(df.c.isNull())

In [ ]:
df.select(df.c, df.a)

In [ ]:
# Create a new column
df.withColumn('upper_c', upper(df.c))

In [ ]:
# Selection on rows
df.filter(df.a == 1)

## Apllying a Function

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [ ]:
df.printSchema()

## Grouping Data

In [ ]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df

In [ ]:
df.groupby('color').avg("v1")

In [ ]:
from pyspark.sql.functions import sum, avg

In [ ]:
df.groupby('color').agg(sum("v2").alias("v2 sum"), avg("v1").alias("v1 avg"))

## Getting Data in/out

In [ ]:
df.write.csv('foo.csv', header=True)

In [ ]:
spark.read.csv('foo.csv', header=True)

In [ ]:
df.write.parquet('bar.parquet')

In [ ]:
spark.read.parquet('bar.parquet')

## Working with SQL

In [ ]:
df

DataFrame and Spark SQL share the same execution engine so they can be interchangeably used seamlessly. For example, you can register the DataFrame as a table and run a SQL easily as below

In [ ]:
df.createOrReplaceTempView("tableA")

In [ ]:
spark.sql("SELECT count(*) from tableA")

In [ ]:
spark.sql("SELECT * from tableA")

In [ ]:
spark.sql("DROP TABLE IF EXISTS tableB")

In [ ]:
spark.sql("CREATE TABLE tableB AS SELECT * FROM tableA")

In [ ]:
spark.sql("SELECT * from tableB")

In [ ]:
spark.sql("DESCRIBE tableB")

In [ ]:
spark.sql("DESCRIBE EXTENDED tableB")

## Pandas on Spark API

In [ ]:
import pyspark.pandas as ps

In [ ]:
import numpy as np

In [ ]:
df.head()

In [ ]:
# from a spark dataframe
psdf = df.pandas_api()
psdf

In [ ]:
type(df)

In [ ]:
type(psdf)

In [ ]:
psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])
psdf

In [ ]:
type(psdf)

### Creating a pyspark.pandas Dataframe from a pandas dataframe

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

In [ ]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
pdf

In [ ]:
type(pdf)

In [ ]:
psdf = ps.from_pandas(pdf)
psdf

In [ ]:
type(psdf)

In [ ]:
psdf.head()

In [ ]:
psdf.describe()

## Grouping data in pandas dataframe

In [ ]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})
psdf

In [ ]:
psdf.groupby('A').sum()

## Plot Data

In [ ]:
psdf.C.plot()

## Getting data in/out (Unsing Pandas API)

In [ ]:
psdf.to_csv('foo2.csv')

In [ ]:
ps.read_csv('foo2.csv').head(10)

In [ ]:
psdf.to_parquet('bar2.parquet')
ps.read_parquet('bar2.parquet').head(10)

In [ ]:
# You can use all the datasources supported py Spark IO
psdf.spark.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)